# Binary Classification Model: Total Trial Fund Usage Prediction

This notebook builds a Logistic Regression model to predict `总体验金使用` (1 = Yes, 0 = No).

## Step 1: Import Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
import joblib
import warnings
from pathlib import Path
from custom_transformers import (
    NumericalImputer, 
    CategoricalImputer, 
    StringImputer, 
    CompletePipeline
)
warnings.filterwarnings('ignore')

## Step 2: Load Data

In [5]:
# Load the dataset - try multiple locations
# Option 1: Check if dataset.xlsx exists in current directory
if Path('dataset.xlsx').exists():
    data_file = Path('dataset.xlsx')
    print(f"Loading data from: {data_file}")
    df = pd.read_excel(data_file)
# Option 2: Check dataset folder
elif Path('dataset').exists() and Path('dataset').is_dir():
    dataset_folder = Path('dataset')
    excel_files = list(dataset_folder.glob('*.xlsx')) + list(dataset_folder.glob('*.xls'))
    if excel_files:
        data_file = excel_files[0]
        print(f"Loading data from: {data_file}")
        df = pd.read_excel(data_file)
    else:
        raise FileNotFoundError("No Excel files found in 'dataset' folder.")
else:
    raise FileNotFoundError(
        "Dataset not found. Please ensure:\n"
        "  1. dataset.xlsx is in the current directory, OR\n"
        "  2. Excel file(s) are in a 'dataset' folder"
    )

print(f"\nDataset shape: {df.shape}")
print(f"\nColumn names: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

Loading data from: dataset.xlsx

Dataset shape: (8585, 30)

Column names: ['user_id', 'partner', 'staff', 'nationality', 'VIP_level', 'spot_level', 'asset_level', '昨日账户余额', '新老用户', '是否kyc', 'ARPU30', 'ARPU7', 'ARPU14', '是否为混合交易用户', '当日合约交易量', '当日合约交易笔数', '当日充值金额', '总合约交易量', '总体验金发放', '合约交易频率', '筛选交易日前7日日均交易笔数', '筛选交易日前7日每笔持仓时间', '筛选交易日前7日平均杠杆倍数', '过去30天平均杠杆值', '当日净充提金额', '是否意向流失', '用户分层', '用户活跃等级', 'user_tag', '总体验金使用']

First few rows:


,user_id,partner,staff,nationality,VIP_level,spot_level,asset_level,昨日账户余额,新老用户,是否kyc,...,筛选交易日前7日日均交易笔数,筛选交易日前7日每笔持仓时间,筛选交易日前7日平均杠杆倍数,过去30天平均杠杆值,当日净充提金额,是否意向流失,用户分层,用户活跃等级,user_tag,总体验金使用
0,1003380027,官网,官网,俄罗斯,0,0,0,36.0,老用户,否,...,0.0,2.0,0.0,38,0,否,中价值,3日活跃用户,主流币_低频低杠杆用户,0
1,1003380027,官网,官网,俄罗斯,0,0,0,36.0,老用户,否,...,11.0,2.0,39.0,38,0,否,中价值,3日活跃用户,长尾币_高频高杠杆用户,0
2,1004534556,Group AF,BD Abdallah,尼日利亚,0,0,0,0.0,老用户,是,...,1.0,2.0,83.0,93,0,否,中价值,3日活跃用户,BTC_ETH_低频高杠杆用户,0
3,1006541013,Group MN,BD Jacob Kasperek,波兰,3,0,0,221.0,老用户,是,...,45.0,5.0,76.0,86,392,否,高价值,3日活跃用户,长尾币_高频高杠杆用户,1
4,1007262247,Group MN,BD Abdulla2,墨西哥,2,0,0,977.0,新用户,否,...,26.0,18.0,26.0,35,0,否,高价值,3日活跃用户,BTC_ETH_高频高杠杆用户,0


## Step 3: Define Column Groups and Preprocessing Pipeline

In [6]:
# Define column groups for preprocessing
target_column = '总体验金使用'

# Numerical columns to fill with 0
numerical_fill_zero = [
    '昨日账户余额', 'ARPU30', 'ARPU7', 'ARPU14',
    '当日合约交易量', '当日合约交易笔数', '当日充值金额',
    '总合约交易量', '总体验金发放', '合约交易频率',
    '筛选交易日前7日日均交易笔数', '筛选交易日前7日每笔持仓时间', 
    '筛选交易日前7日平均杠杆倍数', '过去30天平均杠杆值', '当日净充提金额'
]

# Categorical columns to fill with 0
categorical_fill_zero = ['VIP_level', 'spot_level', 'asset_level']

# Categorical columns to fill with "未持仓用户"
categorical_fill_string = ['user_tag']

# Columns for target encoding
target_encode_columns = [
    'partner', 'staff', 'nationality', '新老用户',
    '是否kyc', '是否为混合交易用户', '是否意向流失',
    '用户分层', '用户活跃等级', 'user_tag'
]

# Extract target variable
if target_column not in df.columns:
    raise ValueError(f"Target column '{target_column}' not found in dataset")

target = df[target_column].copy()
print(f"Target distribution:\n{target.value_counts()}")
print(f"\nTarget value counts (normalized):\n{target.value_counts(normalize=True)}")

# Filter to only columns that exist in the dataset
numerical_fill_zero = [col for col in numerical_fill_zero if col in df.columns]
categorical_fill_zero = [col for col in categorical_fill_zero if col in df.columns]
categorical_fill_string = [col for col in categorical_fill_string if col in df.columns]
target_encode_columns = [col for col in target_encode_columns if col in df.columns]

print(f"\nNumerical columns to fill with 0: {len(numerical_fill_zero)}")
print(f"Categorical columns to fill with 0: {len(categorical_fill_zero)}")
print(f"Categorical columns to fill with '未持仓用户': {len(categorical_fill_string)}")
print(f"Columns for target encoding: {len(target_encode_columns)}")

Target distribution:
总体验金使用
1    5727
0    2858
Name: count, dtype: int64

Target value counts (normalized):
总体验金使用
1    0.667094
0    0.332906
Name: proportion, dtype: float64

Numerical columns to fill with 0: 15
Categorical columns to fill with 0: 3
Categorical columns to fill with '未持仓用户': 1
Columns for target encoding: 10


In [7]:
# Prepare features (remove user_id and target)
features_df = df.drop(columns=[target_column], errors='ignore')
if 'user_id' in features_df.columns:
    # Keep user_id for later but remove from features
    user_ids = features_df['user_id'].copy()
    features_df = features_df.drop(columns=['user_id'])
else:
    user_ids = None

print(f"Feature columns: {features_df.columns.tolist()}")
print(f"\nFeature shape: {features_df.shape}")
print(f"Target shape: {target.shape}")

Feature columns: ['partner', 'staff', 'nationality', 'VIP_level', 'spot_level', 'asset_level', '昨日账户余额', '新老用户', '是否kyc', 'ARPU30', 'ARPU7', 'ARPU14', '是否为混合交易用户', '当日合约交易量', '当日合约交易笔数', '当日充值金额', '总合约交易量', '总体验金发放', '合约交易频率', '筛选交易日前7日日均交易笔数', '筛选交易日前7日每笔持仓时间', '筛选交易日前7日平均杠杆倍数', '过去30天平均杠杆值', '当日净充提金额', '是否意向流失', '用户分层', '用户活跃等级', 'user_tag']

Feature shape: (8585, 28)
Target shape: (8585,)


In [8]:
# Build the preprocessing pipeline using ColumnTransformer
transformers = []

# Numerical imputation (fill with 0)
if numerical_fill_zero:
    transformers.append(('num_imputer', NumericalImputer(fill_value=0), numerical_fill_zero))

# Categorical imputation (fill with 0)
if categorical_fill_zero:
    transformers.append(('cat_imputer_zero', CategoricalImputer(fill_value=0), categorical_fill_zero))

# String imputation (fill with "未持仓用户")
if categorical_fill_string:
    transformers.append(('cat_imputer_string', StringImputer(fill_value='未持仓用户'), categorical_fill_string))

# Target encoding columns (will be handled separately in the pipeline)
# We'll apply target encoding after initial imputation

# Create ColumnTransformer for initial imputation
preprocessor = ColumnTransformer(
    transformers=transformers,
    remainder='passthrough',  # Keep other columns as-is
    verbose_feature_names_out=False
)

print("Preprocessing pipeline created with ColumnTransformer")
print(f"Number of transformers: {len(transformers)}")

Preprocessing pipeline created with ColumnTransformer
Number of transformers: 3


In [9]:
# The CompletePipeline class (imported from custom_transformers) will handle:
# 1. Initial preprocessing (imputation) via ColumnTransformer
# 2. Target encoding (requires y during fit)
# 3. Model training
# 4. Predictions with custom threshold

print("Preprocessing pipeline ready for CompletePipeline")

Preprocessing pipeline ready for CompletePipeline


In [10]:
# CompletePipeline is imported from custom_transformers
# It handles all preprocessing and target encoding automatically
print("CompletePipeline class available from custom_transformers")

CompletePipeline class available from custom_transformers


## Step 4: Split Data (Train, Validation, Test)

In [11]:
# Split data before building pipeline (to avoid data leakage)
# First split: Train+Val (80%) and Test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    features_df, target, test_size=0.2, random_state=42, stratify=target
)

# Second split: Train (64%) and Validation (16%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 5494 samples
Validation set: 1374 samples
Test set: 1717 samples


## Step 5: Build and Train Complete Pipeline

In [12]:
# Build the complete pipeline
model = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
complete_pipeline = CompletePipeline(
    preprocessor=preprocessor,
    target_encode_columns=target_encode_columns,
    model=model,
    threshold=0.6
)

# Fit the pipeline on training data
print("Fitting complete pipeline on training data...")
complete_pipeline.fit(X_train, y_train)
print("Pipeline fitted successfully!")

Fitting complete pipeline on training data...
Pipeline fitted successfully!


## Step 6: Evaluation with Custom Threshold (0.6)

In [13]:
def evaluate_with_threshold(pipeline, X, y, threshold=0.6):
    """
    Evaluate pipeline with custom probability threshold.
    If probability > threshold, predict 1, otherwise 0.
    """
    # Get probabilities
    y_proba = pipeline.predict_proba(X)[:, 1]
    
    # Apply custom threshold
    y_pred = (y_proba > threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    roc_auc = roc_auc_score(y, y_proba)
    
    return accuracy, f1, roc_auc, y_pred, y_proba

In [14]:
# Evaluate on Training set
train_acc, train_f1, train_roc, train_pred, train_proba = evaluate_with_threshold(
    complete_pipeline, X_train, y_train, threshold=0.6
)

print("=== Training Set Results ===")
print(f"Accuracy: {train_acc:.4f}")
print(f"F1-Score: {train_f1:.4f}")
print(f"ROC-AUC Score: {train_roc:.4f}")

=== Training Set Results ===
Accuracy: 0.8759
F1-Score: 0.8996
ROC-AUC Score: 0.9445


## Step 7: Evaluate on Validation and Test Sets

In [15]:
# Evaluate on Validation set
val_acc, val_f1, val_roc, val_pred, val_proba = evaluate_with_threshold(
    complete_pipeline, X_val, y_val, threshold=0.6
)

print("=== Validation Set Results ===")
print(f"Accuracy: {val_acc:.4f}")
print(f"F1-Score: {val_f1:.4f}")
print(f"ROC-AUC Score: {val_roc:.4f}")

=== Validation Set Results ===
Accuracy: 0.8566
F1-Score: 0.8821
ROC-AUC Score: 0.9379


In [16]:
# Evaluate on Test set
test_acc, test_f1, test_roc, test_pred, test_proba = evaluate_with_threshold(
    complete_pipeline, X_test, y_test, threshold=0.6
)

print("=== Test Set Results ===")
print(f"Accuracy: {test_acc:.4f}")
print(f"F1-Score: {test_f1:.4f}")
print(f"ROC-AUC Score: {test_roc:.4f}")

=== Test Set Results ===
Accuracy: 0.8736
F1-Score: 0.8977
ROC-AUC Score: 0.9441


## Step 8: Save Complete Pipeline

In [17]:
# Save the complete pipeline
# The CompletePipeline object contains everything needed for inference
joblib.dump(complete_pipeline, 'trial_fund_model.joblib')
print("Complete pipeline saved as 'trial_fund_model.joblib'")
print("\nPipeline includes:")
print("- Numerical imputation (fill with 0)")
print("- Categorical imputation (fill with 0 for VIP_level, spot_level, asset_level)")
print("- String imputation (fill with '未持仓用户' for user_tag)")
print("- Target encoding for categorical features")
print("- Logistic Regression model")
print("- Custom threshold (0.6) for predictions")

Complete pipeline saved as 'trial_fund_model.joblib'

Pipeline includes:
- Numerical imputation (fill with 0)
- Categorical imputation (fill with 0 for VIP_level, spot_level, asset_level)
- String imputation (fill with '未持仓用户' for user_tag)
- Target encoding for categorical features
- Logistic Regression model
- Custom threshold (0.6) for predictions


## Summary

The model has been trained and saved. The preprocessing pipeline includes:
- Target encoding for categorical variables
- Imputation rules for missing values
- Logistic Regression model
- Custom threshold (0.6) for predictions

Use the inference script to make predictions on new Excel files.